In [1]:
%load_ext autoreload
%autoreload 2

import torch
import seqm
import importlib

from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure

# importlib.reload(seqm.ElectronicStructure)

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
    

Decorating your function! <function KSA_XL_BOMD.one_step at 0x7fd5707f53f0>


In [22]:
%%time

### create molecule object:
species = torch.as_tensor([[8,6,1,1]], dtype=torch.int64, device=device)

coordinates = torch.tensor([
                            [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                            ]
                            ], device=device)

print('Coordinates shape', coordinates.shape)
print('Species shape', species.shape)
const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecules = seqm.Molecule.Molecule(const, seqm_parameters, coordinates, species).to(device)

print(molecules)

Coordinates shape torch.Size([1, 4, 3])
Species shape torch.Size([1, 4])
Molecule(
  (const): Constants()
  (parser): Parser()
)
CPU times: user 4.28 ms, sys: 7.44 ms, total: 11.7 ms
Wall time: 11.9 ms


In [24]:
### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)

tore Parameter containing:
tensor([0., 1., 0., 1., 2., 3., 4., 5., 6., 7., 0., 1., 2., 3., 4., 5., 6., 7.,
        0.], device='cuda:0')
qn Parameter containing:
tensor([0., 1., 0., 2., 2., 2., 2., 2., 2., 2., 0., 3., 3., 3., 3., 3., 3., 3.,
        0.], device='cuda:0')


In [ ]:
print()

In [9]:
xis = torch.arange(0, 21, dtype=torch.float64, device=device)

In [7]:
xis.size(0)

21

In [3]:
tri_idx = torch.triu_indices(int(6), int(6))


In [6]:
xis_tri = torch.zeros((6, 6), device=device)

In [4]:
tri_idx

tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5],
        [0, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 2, 3, 4, 5, 3, 4, 5, 4, 5, 5]])

In [17]:
xis_tri = torch.zeros((6, 6), device=device)


In [10]:
xis_tri[tri_idx[0], tri_idx[1]] = xis

In [13]:
tri_idx.shape

torch.Size([2, 21])

In [11]:
xis_tri

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [ 0.,  6.,  7.,  8.,  9., 10.],
        [ 0.,  0., 11., 12., 13., 14.],
        [ 0.,  0.,  0., 15., 16., 17.],
        [ 0.,  0.,  0.,  0., 18., 19.],
        [ 0.,  0.,  0.,  0.,  0., 20.]], device='cuda:0')

In [18]:
 xis_full = xis_tri + xis_tri.T - torch.diag(torch.diag(xis_tri))

In [24]:
 xis_full.shape

torch.Size([6, 6])

In [19]:
 xis_full == xis_full.T

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True]], device='cuda:0')

In [31]:
from seqm.seqm_functions.pack import unpack

In [ ]:
pack.packone()

In [27]:
P = pack.unpackone(xis_full, 1, 2, 12)

In [28]:
P

tensor([[ 0.,  1.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  6.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 2.,  7.,  0.,  0.,  0., 11.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
       device='cuda:0')

In [30]:
P.shape

torch.Size([12, 12])

In [117]:
M = torch.tensor([[[    -0.00000,      1.47817,     -0.00000,     -0.00000,     -0.00000,      0.00000,      0.00000,      0.00000,     -0.00000,      0.00000,      0.00000,      0.00000],
         [     1.47817,     -0.00000,      0.00000,      2.92233,     -2.74748,      0.00000,      0.00000,      0.00000,     -2.74748,      0.00000,      0.00000,      0.00000],
         [    -0.00000,      0.00000,     -0.00000,     -0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [    -0.00000,      2.92233,     -0.00000,     -0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [    -0.00000,     -2.74748,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [     0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [     0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [     0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [    -0.00000,     -2.74748,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,     -0.00000,      0.00000,      0.00000,      0.00000],
         [     0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [     0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
         [     0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000]]])

In [119]:
M.shape

torch.Size([1, 12, 12])

In [121]:
nHeavy = torch.tensor([1])
nHydro = torch.tensor([2])

In [122]:
P = pack.pack(M, nHeavy, nHydro)

In [123]:
P

tensor([[[-0.00000,  1.47817, -0.00000, -0.00000, -0.00000, -0.00000],
         [ 1.47817, -0.00000,  0.00000,  2.92233, -2.74748, -2.74748],
         [-0.00000,  0.00000, -0.00000, -0.00000,  0.00000,  0.00000],
         [-0.00000,  2.92233, -0.00000, -0.00000,  0.00000,  0.00000],
         [-0.00000, -2.74748,  0.00000,  0.00000,  0.00000,  0.00000],
         [-0.00000, -2.74748,  0.00000,  0.00000,  0.00000, -0.00000]]])

In [45]:
values = [
    0.00000, 1.47797, 0.00000, -0.00000, 0.00000, 0.00000, -0.00000, 2.92263,
    -0.00000, 0.00000, -0.00000, -2.74742, 0.00000, 0.00000, 0.00000, 0.00000,
    -2.74742, 0.00000, 0.00000, -0.00000, 0.00000
]

tensor = torch.tensor(values)

print(tensor)

tensor([ 0.0000,  1.4780,  0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  2.9226,
        -0.0000,  0.0000, -0.0000, -2.7474,  0.0000,  0.0000,  0.0000,  0.0000,
        -2.7474,  0.0000,  0.0000, -0.0000,  0.0000])


In [60]:
values

[0.0,
 1.47797,
 0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 2.92263,
 -0.0,
 0.0,
 -0.0,
 -2.74742,
 0.0,
 0.0,
 0.0,
 0.0,
 -2.74742,
 0.0,
 0.0,
 -0.0,
 0.0]

In [98]:
# tri_idx = torch.triu_indices(int(6), int(6), offset=0)
eta = torch.zeros((6, 6), device='cpu') 

l = 0
for i in range(0, 6):
    for j in range(i):
        l += 1
        print('j', j)
        eta[i,j] += values[l-1]
        eta[j,i] += values[l-1]
    l += 1 
    eta[i,i] += values[l-1]
# l = 0       
# for i in range(0, 6):      
        
#         
#xis_tri[tri_idx[0], tri_idx[1]] = tensor

j 0
j 0
j 1
j 0
j 1
j 2
j 0
j 1
j 2
j 3
j 0
j 1
j 2
j 3
j 4


In [99]:
eta

tensor([[ 0.0000,  1.4780,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.4780,  0.0000,  0.0000,  2.9226, -2.7474, -2.7474],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  2.9226,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -2.7474,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -2.7474,  0.0000,  0.0000,  0.0000,  0.0000]])

In [309]:
 sym = torch.tensor([[0.00000,    1.47797,   -0.00000,   -0.00000,   -0.00000,   0.00000],   
                    [1.47797,    0.00000,    0.00000,   2.92263,   -2.74742,   -2.74742],   
                    [-0.00000,    0.00000,    0.00000,   -0.00000,    0.00000,    0.00000],   
                    [-0.00000,    2.92263,   -0.00000,    0.00000,    0.00000,    0.00000],   
                    [-0.00000,   -2.74742,    0.00000,    0.00000,    0.00000,   -0.00000],   
                    [0.00000,   -2.74742,    0.00000,    0.00000 ,  -0.00000,    0.00000]])

In [310]:
antisym = torch.tensor([[0.00000,   -0.19859,   -0.00000,    0.00000,    0.00000,    0.00000],   
                        [3.15453,    0.00000,    0.00000,    6.19148,   -5.49484,   -5.49484],   
                        [-0.00000,   -0.00000,    0.00000,   -0.00000,    0.00000,    0.00000],   
                        [-0.00000,   -0.34623,   -0.00000,    0.00000,    0.00000,    0.00000],   
                        [-0.00000,    0.00000,   -0.00000,   -0.00000,    0.00000,    0.00000],   
                        [0.00000,    0.00000,   -0.00000,    0.00000,   -0.00000,    0.00000]])

In [311]:
antisym - sym

tensor([[ 0.00000, -1.67656,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 1.67656,  0.00000,  0.00000,  3.26885, -2.74742, -2.74742],
        [ 0.00000, -0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -3.26886,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000,  2.74742, -0.00000, -0.00000,  0.00000,  0.00000],
        [ 0.00000,  2.74742, -0.00000,  0.00000,  0.00000,  0.00000]])

In [325]:
ar = torch.tensor([0.00000, 1.47797, 0.00000, -0.00000, 0.00000, 0.00000, -0.00000   ,2.92263   ,-0.00000   ,0.00000   ,-0.00000   ,-2.74742   , 0.00000   , 0.00000    ,0.00000    ,0.00000   ,-2.74742    ,0.00000    ,0.00000   ,-0.00000   ,0.00000])

In [344]:
ar.size()

torch.Size([21])

In [345]:
matrix = torch.zeros(6,6)

matrix2 = torch.zeros(6,6)

In [346]:
l = 0
for i in range(0, 6): # TODO" vectorize
    for j in range(0,i):
        l += 1
        matrix[i,j] += ar[l-1]
        matrix[j,i] += ar[l-1]
    l += 1 
    matrix[i,j] += ar[l-1]

In [332]:
print(matrix)

tensor([[ 0.00000,  1.47797,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 1.47797,  0.00000,  0.00000,  2.92263, -2.74742, -2.74742],
        [ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000,  2.92263,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -2.74742,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -2.74742,  0.00000,  0.00000,  0.00000,  0.00000]])


In [333]:
l = 0
for i in range(0, 6): # TODO" vectorize
    for j in range(0,i):
        l += 1
        matrix2[i,j] += ar[l-1]
        matrix2[j,i] -= ar[l-1]
    l += 1 

In [334]:
matrix2

tensor([[ 0.00000, -1.47797,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 1.47797,  0.00000,  0.00000, -2.92263,  2.74742,  2.74742],
        [ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000,  2.92263,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -2.74742,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -2.74742,  0.00000,  0.00000,  0.00000,  0.00000]])

In [336]:
matrix + matrix2

tensor([[ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 2.95594,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000,  5.84526,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -5.49484,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -5.49484,  0.00000,  0.00000,  0.00000,  0.00000]])

In [340]:
values = [0.00000, 0.83828, 0.00000, -0.00000, -0.00000, 0.00000, -0.00000, -1.63443, 0.00000, 0.00000, -0.00000,
          1.37371, -0.00000, -0.00000, 0.00000, 0.00000, 1.37371, -0.00000, -0.00000, -0.00000, 0.00000]

# Convert the list to a Torch tensor
tensor = torch.tensor(values)

print(tensor)






tensor([ 0.00000,  0.83828,  0.00000, -0.00000, -0.00000,  0.00000, -0.00000, -1.63443,  0.00000,  0.00000, -0.00000,  1.37371, -0.00000, -0.00000,  0.00000,  0.00000,  1.37371, -0.00000, -0.00000,
        -0.00000,  0.00000])


In [341]:
tensor.size()

torch.Size([21])

In [347]:
l=0
matrix3 = torch.zeros(6,6)
for i in range(0, 6): # TODO" vectorize
    for j in range(0,i):
        l += 1
        matrix3[i,j] += tensor[l-1]
        matrix3[j,i] -= tensor[l-1]
    l += 1 


In [348]:
matrix3

tensor([[ 0.00000, -0.83828,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.83828,  0.00000,  0.00000,  1.63443, -1.37371, -1.37371],
        [ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000, -1.63443,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000,  1.37371,  0.00000,  0.00000,  0.00000,  0.00000],
        [ 0.00000,  1.37371,  0.00000,  0.00000,  0.00000,  0.00000]])

In [365]:
# Input tensor
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Antisymmetrize the tensor along dimensions 0 and 1
antisym_tensor = 0.5 * (torch.einsum('ij->ij', tensor) - torch.einsum('ji->ij', tensor))

print(antisym_tensor)

tensor([[ 0., -1., -2.],
        [ 1.,  0., -1.],
        [ 2.,  1.,  0.]])


In [361]:
ind = torch.tril_indices(3, 3)

In [362]:
ind

tensor([[0, 1, 1, 2, 2, 2],
        [0, 0, 1, 0, 1, 2]])

In [366]:
tensor

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [367]:
tensor[ind[0]] = 0

In [368]:
tensor

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [355]:
tensor

tensor([[-1, -2, -3],
        [-4, -5, -6],
        [-7, -8, -9]])

In [371]:
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Get the lower triangle indices
rows, cols = torch.tril_indices(tensor.shape[0], tensor.shape[1], offset=0)

# Multiply the lower triangle elements by -1
tensor[rows, cols] *= -1

print(tensor)

tensor([[-1,  2,  3],
        [-4, -5,  6],
        [-7, -8, -9]])


In [372]:
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Get the lower triangle indices
rows, cols = torch.tril_indices(tensor.shape[0], tensor.shape[1])

# Multiply the lower triangle elements by -1
tensor[rows, cols] *= -1

# Add the diagonal elements back
tensor[torch.eye(tensor.shape[0]).bool()] *= -1

print(tensor)

tensor([[ 1,  2,  3],
        [-4,  5,  6],
        [-7, -8,  9]])


In [373]:
matrix = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]], dtype=torch.float32)

# Compute the symmetric part
symmetric = 0.5 * (matrix + matrix.t())

# Compute the antisymmetric part
antisymmetric = 0.5 * (matrix - matrix.t())

print("Original Matrix:")
print(matrix)

print("Symmetric Part:")
print(symmetric)

print("Antisymmetric Part:")
print(antisymmetric)

Original Matrix:
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]], dtype=torch.float32)
Symmetric Part:
tensor([[1., 3., 5.],
        [3., 5., 7.],
        [5., 7., 9.]], dtype=torch.float32)
Antisymmetric Part:
tensor([[ 0., -1., -2.],
        [ 1.,  0., -1.],
        [ 2.,  1.,  0.]], dtype=torch.float32)


In [374]:
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Set the diagonal elements to 0
tensor.diagonal().fill_(0)

print(tensor)

tensor([[0, 2, 3],
        [4, 0, 6],
        [7, 8, 0]])


In [375]:
import torch

# Example matrix
M = torch.tensor([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])

# Calculate the symmetric matrix
symmetric_matrix = 0.5 * (M + M.T)

print(symmetric_matrix)

tensor([[1., 3., 5.],
        [3., 5., 7.],
        [5., 7., 9.]])
